# Hogbom Deconvolve Demonstrator

This notebook demonstrates the implementation of Hogbom clean within the astroviper framework. 
CASA is used to generate the initial set of images (residual, PSF). The CASA task `deconvolve` is run to deconvolve the residual image. astroviper Hogbom is also run on a copy of the same images, and the two outputs are compared. 

This notebook is organized into three sections : The first section runs the CASA deconvolution, the second the astroviper deconvolution, and finally the comparison images and plots. 

This has to be done, because importing CASA alongside xradio causes some namespace conflicts and segmentation faults, so once we are done with all the CASA functionality, we will import the astroviper libraries.

In [ ]:
# Common imports across all sections

import os
import ssl
import certifi
import urllib
import tarfile 
import glob
import shutil
import numpy as np
import toolviper

## Download images via toolviper

Wipe any images with the same name on disk, and pull the images from the toolviper data repo in order to make the comparison between astroviper hogbom clean and CASA hogbom clean

In [ ]:
def wipe_images(imlist):
    for im in imlist:
        if os.path.exists(im):
            shutil.rmtree(im)

In [ ]:
resid_orig = "test_hogbom_multisrc.residual"
psf_orig = "test_hogbom_multisrc.psf"
casa_deconv_resid = "casa_hogbom_deconv.residual"

wipe_images([resid_orig, psf_orig, casa_deconv_resid])

# Update data manifest always
toolviper.utils.data.update()
toolviper.utils.data.download([resid_orig, psf_orig, casa_deconv_resid])

## NOTE

The progress bars on the toolviper download may not appear complete, which is a bug in the Jupyter display. The files are very small (< 1 MB total) and the download completes relatively rapidly.

## Astroviper : Load images, run deconvolution

Import the astroviper and xradio libraries, load the copies of the residual and PSF images generated by CASA, and run deconvolution.

In [ ]:
# Import astroviper libraries
try:
    from xradio.image import load_image
    from astroviper.core.imaging import deconvolution
except ImportError:
    print("Please install the astroviper libraries. This can be done via pip install astroviper, "
          "or cloning the git repository (https://github.com/casangi/astroviper/) and installing "
          "the relevant branch.")


resid_xds = load_image(resid_orig)
psf_xds = load_image(psf_orig)
casa_residual_xds = load_image(casa_deconv_resid)

In [ ]:
ret, model_image, residual_image = deconvolution.hogbom_clean(resid_xds, psf_xds, deconv_params={'gain':0.1, 'niter':500, 'threshold':0})

In [ ]:
print(ret)

In [ ]:
print(np.squeeze(residual_image['SKY'].data).max())

## Results

Compare the results of the astroviper and CASA outputs. Residual images after deconvolution, model images, fractional differences.

In [ ]:
try:
    import holoviews as hv
except ImportError:
    print(f"Please install holoviews. pip install holoviews")
    exit
    
hv.extension('bokeh')

In [ ]:
img1 = hv.Image(np.squeeze(residual_image['SKY'].values)).opts(tools=['hover'], title='Astroviper residual image', 
                                        width=300, height=300, colorbar=True)
img2 = hv.Image(np.squeeze(np.squeeze(casa_residual_xds['SKY'].values))).opts(tools=['hover'], title='CASA residual image', 
                                        width=300, height=300, colorbar=True)

diff = np.squeeze(casa_residual_xds['SKY'].values) - np.squeeze(residual_image['SKY'].values)
img3 = hv.Image(diff).opts(tools=['hover'], title='Difference residual image', 
                                        width=500, height=500, colorbar=True)

In [ ]:
grid = hv.Layout([img1, img2])
grid

In [ ]:
img3